<p>
<font size='5' face='Georgia, Arial'>IIC2233 Apunte Programación Avanzada</font><br>
<font size='1'>&copy; 2015 Karim Pichara - Christian Pieringer. Todos los derechos reservados.</font>
<br>
<font size='1'> Modificado desde 2017-2 al 2025-2 por Equipo Docente IIC2233</font>
</p>

# Tabla de contenidos

1. [Decoradores](#decoradores)
    1. [Objetos de primera clase](#objetos-de-primera-clase)
    2. [Sintaxis de los decoradores](#sintaxis-de-los-decoradores)
    3. [Funciones dentro de funciones](#funciones-dentro-de-funciones)
2. [Ejemplos de decoradores más usados](#ejemplos-de-decoradores-mas-usados)
    1. [Properties: @property](#properties-property)

        1. [Ejemplo: Definiendo una clase `Email`](#ejemplo-definiendo-una-clase-email)
    
    2. [Métodos estáticos](#métodos-estáticos)

# Decoradores

Conceptualmente, un decorador es en sí una función ([callable](https://docs.python.org/3.11/c-api/call.html) o invocable) que toma otra función como *input* y retorna una versión modificada de esa función. Los decoradores proveen una forma sintética de agregar funcionalidades a una función, sin cambiar las líneas de código que la componen en su origen. Se puede modificar o aumentar el comportamiento de una función, o incluso una clase. 

## Objetos de primera clase

Una de las razones de su uso ampliado, que veremos a lo largo del curso, es que las funciones en Python son **objetos de primera clase**. Esto implica que:

- Puedes asignar una función a una variable como si fuera un *int* o un *string*.
- Puedes pasar una función como argumento a otra función (o devolverla como resultado).
- Puedes almacenar funciones en estructuras de datos (listas, diccionarios, etc.).
- Tienen atributos (puedes agregar o acceder dinámicamente a los atributos de una función).

Veamos un ejemplo de funciones como objetos de primera clase en Python.

In [1]:
# función regular que espera un nombre como string y retorna un string
def decir_hola(nombre: str) -> str:
    return f"Hola {nombre}"

# función regular que espera un nombre como string y retorna un string
def ser_bacan(nombre: str) -> str:
    return f"Oye {nombre}, ¡juntos somos demasiado bacanes!"

# función que espera una función como argumento y retorna un string
def saludar_dani(funcion_saludadora) -> str:
    return funcion_saludadora("Dani")

In [2]:
print(saludar_dani(decir_hola))
print(saludar_dani(ser_bacan))

Hola Dani
Oye Dani, ¡juntos somos demasiado bacanes!


Tenga en cuenta que `saludar_dani(decir_hola)` hace referencia a dos funciones, `saludar_dani()` y `decir_hola`, pero de forma diferente. La función `decir_hola` se nombra sin paréntesis. Esto significa que solo se pasa una **referencia** a la función, y la función no se ejecuta. La función `saludar_dani()`, en cambio, se escribe con paréntesis, por lo que se llamará normalmente.

Esta es una distinción importante, crucial para el funcionamiento de las funciones como objetos de primera clase. Un nombre de función sin paréntesis es una referencia a una función, mientras que un nombre de función con paréntesis finales la llama y hace referencia a su valor de retorno.

In [3]:
def decir_hola(nombre: str) -> None:
    print(f"Hola {nombre}")

saludar = decir_hola # asignamos la función a una variable
saludar("Dani")

Hola Dani


En el caso del código de arriba, `decir_hola` es simplemente otra referencia (o puntero) al mismo objeto de función al que se refiere `saludar`. En el ejemplo a continuación, vemos una situación similar, pero ahora ejemplificando que una función puede perfectamente retornar otra función.

In [4]:
# función regular que espera un nombre como string y retorna un string
def decir_hola(nombre: str) -> None:
    return f"Hola {nombre}"

# función que espera una función como argumento y retorna una función
def saludar_dani(funcion_saludadora):
    return funcion_saludadora

saludar = saludar_dani(decir_hola)
print(saludar("Dani"))

Hola Dani


Al ser objetos, los decoradores pueden recibir funciones como argumentos, inspeccionarlas o modificarlas, y devolverlas (o devolver nuevos objetos función). Esta es la idea central que hace posibles los decoradores. Resumamos lo aprendido hasta ahora:

- Los decoradores son funciones (callables o invocables) que toman un objeto de función como entrada.
- Pueden añadir atributos, encapsular comportamientos o cualquier otra función, ya que la entrada es simplemente otro objeto de Python.
- Devuelven la función original o una versión modificada de esta.

Por lo tanto, en Python, tratar las funciones como objetos es lo que da lugar a construcciones potentes como decoradores.

## Sintaxis de los decoradores

En Python, los decoradores se aplican comúnmente usando el símbolo @, continuado del nombre del decorador, de esta forma: `@nombre_decorador`. Esta se coloca en la línea directamente arriba de la definición de la función que será "decorada".

Cuando vemos este código:

```python
@decorador
def mi_funcion():
    pass
```

Es realmente una abreviación para:

```python
def mi_funcion():
    pass
    
mi_funcion = decorador(mi_funcion)
```

En otras palabras, la sintaxis @decorador significa que `decorador` recibe a `mi_funcion` como parámetro, y lo que sea que `decorador(mi_funcion)` returne se convertirá en el nuevo valor de `mi_funcion`.

Veamos el siguiente ejemplo, que modifica un string agregando un signo de exclamación al final de este.

In [5]:
def agregar_exclamacion(func):
    def decorada():
        return func() + "!"
    return decorada

@agregar_exclamacion
def decir_hola():
    return "Hola"

print(decir_hola())

Hola!


Paso a paso:
1. `agregar_exclamacion` toma una función (func) y devuelve una nueva función (`decorada`).
2. La ​​nueva función (`decorada`) llama a la función original y añade `"!"` al resultado.
3. El decorador `@agregar_exclamacion` se aplica a `decir_hola`, por lo que al llamar a `decir_hola()`, modifica la salida añadiendo `"!"`.

Lo que logramos: Tomar una función y cambiar ligeramente su comportamiento sin cambiar su código original.



## Funciones dentro de funciones

Notarás que en el ejemplo anterior, estamos definiendo una función dentro de otra. ¿Acaso no estaba prohibido en intro? 🤔

**¿Por qué es válido escribir una función dentro de otra función en Python?** En Python, escribir una función dentro de otra función es válido por la misma razón anterior de que las funciones son objetos de primera clase. Esto significa que:

- Las funciones pueden definirse en cualquier lugar, incluso dentro de otras funciones.
- Las funciones internas (también llamadas funciones anidadas) solo existen dentro del ámbito de la función externa, lo que ayuda a mantener el código organizado y evita funciones globales innecesarias.

**¿Por qué es importante esto para los decoradores?** La función interna modifica el comportamiento de la función original, manteniéndola separada del ámbito global. Esta capacidad de definir y devolver funciones dinámicamente es lo que hace que Python sea flexible y potente para aplicaciones como decoradores.

In [6]:
def funcion_externa():
    def funcion_interna():
        print("Hello darkness my old friend!")
    
    # Llamamos a la funcion interna dentro de la externa
    funcion_interna()  

funcion_externa()

Hello darkness my old friend!


Aquí, `funcion_interna()` solo existe dentro de `funcion_externa()` y no se puede llamar desde afuera. ¡Probemos transformando esta misma dinámica usando la sintaxis de decoradores!

In [7]:
def mi_decorador(func):
    def funcion_interna():
        print("Hello darkness my old friend!")
        func()
    return funcion_interna  # Return the modified function

@mi_decorador
def saludar():
    print("I've come to talk with you again")

saludar()

Hello darkness my old friend!
I've come to talk with you again


# Ejemplos de decoradores mas usados

Una de las razones más importantes para enseñar decoradores en este curso, es porque es una muy buena forma de programar, y es parte del conjunto de buenas prácticas que actualmente se usan en la industria. Permite evitar la duplicación de código extrayendo la lógica repetida en decoradores reutilizables. Podrán ver aplicaciones inmediatas en áreas como desarrollo web, servicios web, seguridad, pruebas (*testing*) y registro (*logging*).

## *Properties*: `@property`

En los lenguajes OOP, existen formas de definir atributos de clase _públicos_ o _privados_. Un atributo privado se considera como un atributo que no quisiéramos modificar fácilmente, es decir, que tenga cierta protección en su lectura o escritura. En Python, **todos** los atributos y métodos de una clase son **públicos**, y el hecho de iniciar el nombre de un atributo o método con _underscore_ (guión bajo) es una convención (y una buena práctica), pero no asegura un caracter privado de estos elementos.

Una consecuencia de tener atributos privados (o casi privados) es que si queremos modificarlos tenemos que, forzosamente, utilizar un método. En el paradigma OOP, se definen métodos específicos para **obtener el valor de un atributo (privado)**, y para **actualizar el valor de un atributo (privado)**. A estos métodos se llama respectivamente **getters** y **setters**.

`property(get, set, del)` es una función integrada (built-in) de Python que crea un atributo administrado en una clase. Permite definir un *getter*, un *setter* y un *deleter* para un atributo de forma controlada. Puedes encontrar más información en la [documentación de la función](https://docs.python.org/3/library/functions.html#property).

La función se utiliza de la siguiente forma:
```python
property(fget=None, fset=None, fdel=None)
```
Para esta función especial de Python, vemos que lo que pide como argumento son funciones.
- fget: Una función que actuará como getter (va a buscar el atributo).
- fset: Una función que actuará como setter (setea el atributo).
- fdel: Una función que actuará como deleter (borrará el atributo).

Veamos el siguiente ejemplo donde trataremos de proteger la informacion de una instancia de la clase `Circulo`.

In [8]:
class Circulo:
    
    def __init__(self, radio):
        self._radio = radio  # atributo a proteger, privado

    # Getter para el radio
    def get_radio(self):
        print("Obteniendo el radio...")
        return self._radio

    # Setter para el radio, que asegura valores positivos
    def set_radio(self, value):
        if value >= 0:
            self._radio = value
        else:
            print("El radio no puede ser negativo")
    
    # Metodo que retorna el area del circulo
    def area(self):
        print("Obteniendo el área...")
        print("Area:",3.14 * self._radio ** 2)

    # Definimos propiedades usando property()
    radio = property(get_radio, set_radio)

c = Circulo(5)
print("Radio:", c.radio)
c.area()

# Intentamos cambiar el radio a un valor negativo
c.radio = -3
c.area()

# Cambiamos el radio a un valor válido
c.radio = 10
c.area()

Obteniendo el radio...
Radio: 5
Obteniendo el área...
Area: 78.5
El radio no puede ser negativo
Obteniendo el área...
Area: 78.5
Obteniendo el área...
Area: 314.0


Como vemos, la función *built-in* `property` de Python recibe a otras funciones como argumentos. Esto la hace ideal para utilizar los decoradores. El decorador @property en Python se usa para definir un método como un atributo de solo lectura. Esto permite que una clase exponga un método como si fuera un atributo normal, sin necesidad de paréntesis al acceder a él. Veamos el mismo ejemplo, ahora usando el decorador @property. Además, agregaremos una segunda propiedad, el área.

In [9]:
class Circulo:

    def __init__(self, radio):
        self._radio = radio  # Atributo privado, queremos protegerlo

    @property
    def radio(self):
        # Getter para el radio
        print("Obteniendo el radio...")
        return self._radio

    @radio.setter
    def radio(self, valor):
        # Setter para el radio, asegurando valores positivos
        if valor >= 0:
            self._radio = valor
        else:
            print("El radio no puede ser negativo")

    @property
    def area(self):
        # Método que retorna el área del círculo (atributo de solo lectura)
        print("Obteniendo el área...")
        return 3.14 * self._radio ** 2

c = Circulo(5)
print("Radio:", c.radio)
print("Área:", c.area) # ahora es un atributo

# Intentamos cambiar el radio a un valor negativo
c.radio = -3 
print("Área:", c.area)

# Cambiamos el radio a un valor válido
c.radio = 10
print("Área:", c.area)

Obteniendo el radio...
Radio: 5
Obteniendo el área...
Área: 78.5
El radio no puede ser negativo
Obteniendo el área...
Área: 78.5
Obteniendo el área...
Área: 314.0


### Ejemplo: definiendo una clase `Email`

Este ejemplo utiliza la segunda forma de usar *properties*, que es definiendo los métodos y luego asignarlos a una variable usando `property`. 

In [10]:
class Email:
    
    def __init__(self, address):
        self._email = address
        
    def _get_email(self):
        return self._email
        
    def _set_email(self, value):
        if '@' not in value:
            print(f"El string {value} no parece una dirección de correo.")
        else:
            self._email = value 

    email = property(_get_email, _set_email)

In [11]:
mail = Email("profe1@gmail.com")
print(mail.email)
mail.email = "profe2@gmail.com"
print(mail.email)
mail.email = "profe2.com"

profe1@gmail.com
profe2@gmail.com
El string profe2.com no parece una dirección de correo.


La siguiente versión de la clase `Email` define la `property` utilizando la notación de decoradores. Es equivalente al ejemplo anterior, y su funcionamiento es igualmente equivalente.

In [12]:
class Email2:
    
    def __init__(self, address):
        self._email = address
    
    @property
    def email(self):
        return self._email
    
    @email.setter
    def email(self, value):
        if '@' not in value:
            print(f"El string {value} no parece una dirección de correo.")
        else:
            self._email = value


In [13]:
mail = Email2("profe1@gmail.com")
print(mail.email)
mail.email = "profe2@gmail.com"
print(mail.email)
mail.email = "profe2.com"

profe1@gmail.com
profe2@gmail.com
El string profe2.com no parece una dirección de correo.


## Métodos estáticos

Imaginemos que nos solicitan crear una clase para poder sumar y restar números. Para esta solicitud, cramos una clase `MiMath` que permita realizar dichas operaciones.

In [14]:
class MiMath:

    def sumar(self, num1, num2):
        return num1 + num2

    def restar(self, num1, num2):
        return num1 - num2

In [15]:
print(MiMath().sumar(1, 2))
print(MiMath().restar(5, 3))

3
2


Si bien la clase anterior cumple con lo solicitado, para PEP8 no es correcto hacer clases que tengan `self` cuando es necesario. Una solución a esto es utilizar funciones (`def sumar(num1, num2)`), pero en otros lenguajes que sean extrictamente OOP, como Java, no es posible crear funciones que no esten adscritas a una clase, sino que debemos crear una clase con dicho métodos. La solución para crear métodos que no necesitan al `self` son **métodos estáticos**.

Los métodos estáticos (_static methods_) son métodos que pertenecen a una clase. No obstante, no dependen de información de la clase ni de atributos de la instancia. Por lo tanto, en este caso no se necesita ningún atributo principal como sucede con los métodos normales, es decir, no requiere del `self`.

Para declarar un método estático, solo necesitamos usar el decorador `@staticmethod` y nuestro método sin el `self`.

In [16]:
class MiMath:

    @staticmethod
    def sumar(num1, num2):
        return num1 + num2

    @staticmethod
    def restar(num1, num2):
        return num1 - num2

In [17]:
print(MiMath.sumar(5, 6))
print(MiMath.restar(14, 1))

11
13


Como puedes notar, en estos ejemplos no fue necesario generar una instancia de la clase `MiMath`, sino que solo se hace `clase.Metodo`. Esto es porque no necesitamos de usar `self` en los métodos usados. Esta forma nos permite generar métodos dentro de una clase, lo cual permite tener empaquetadas ciertas funcionalidades 

Tambien, resulta útil tener un indicador de que ciertos métodos no requieren del `self` y por lo tanto, no accederán ni modificarán el estado de la instancia. Es cierto que en Python se podría hacer lo mismo con un método de instancia como el ejemplo adicional o utilizar funciones, pero a veces el lenguaje no permite dichas opciones.